# **Análisis de distancias semánticas entre términos GO de Proteínas No Moonlighting**
Este notebook explora las distancias semánticas entre términos GO asociados con proteínas candidatas a no moonlighting (selecionadas masivamente de UniProt). Calcularemos el promedio y el máximo de distancias semánticas entre pares de términos GO dentro de las categorías de Cellular Component (CC), Molecular Function (MF), y Biological Process (BP) para cada proteína.


## Montaje de Google Drive
Montamos Google Drive para acceder y guardar archivos directamente desde este entorno.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preparación del entorno de análisis
Cargamos el conjunto de datos de proteínas candidatas a no moonlighting para realizar nuestro análisis.

In [ ]:
import pandas as pd

all_proteins_metrics = pd.read_csv('/content/drive/My Drive/all_proteins_metrics.csv')
all_proteins_metrics.columns = all_proteins_metrics.columns.str.strip()

# Visualizamos las columnas del dataset.
all_proteins_metrics.columns

Index(['accessioncode', 'proteinentry', 'evidencecode', 'organism',
       'go_term_1_id', 'category1', 'description1', 'informationcontent1',
       'go_term_2_id', 'category2', 'description2', 'informationcontent2',
       'resnikdistance', 'minimumbranchlength', 'semanticdistance',
       'semanticsimilarity'],
      dtype='object')

## Definición de funciones para el cálculo de estadísticas
Definimos una función que calculará el promedio y el máximo de la distancia semántica entre términos GO. Además, esta función identificará los pares de términos GO que presentan la mayor distancia semántica dentro de cada categoría para cada proteína.


In [ ]:
def calculate_stats_and_unique_pairs(group):
    avg_mbl = group['minimumbranchlength'].mean()
    max_mbl = group['minimumbranchlength'].max()

    # Identificamos y extraemos los pares de términos con distancia máxima, evitando duplicados.
    max_pairs = group[group['minimumbranchlength'] == max_mbl][['go_term_1_id', 'go_term_2_id']].drop_duplicates()
    max_pairs_list = max_pairs.apply(lambda x: (x['go_term_1_id'], x['go_term_2_id']), axis=1).tolist()

    return pd.Series({
        'Avg_MBL': avg_mbl,
        'Max_MBL': max_mbl,
        'GO_Pairs_Max_MBL': max_pairs_list
    })

## Cálculo y combinación de estadísticas por categoría
Agrupamos los datos por 'accesioncode' (UniProt ID) y por categoría de término GO para calcular las estadísticas de distancia semántica.


In [ ]:
results_category1 = all_proteins_metrics.groupby(['accessioncode', 'category1']).apply(calculate_stats_and_unique_pairs).rename_axis(['UniProt IDs', 'Category']).reset_index()
results_category2 = all_proteins_metrics.groupby(['accessioncode', 'category2']).apply(calculate_stats_and_unique_pairs).rename_axis(['UniProt IDs', 'Category']).reset_index()

# Combinamos ambos resultados.
results = pd.concat([results_category1, results_category2])

## Consolidación de pares y cálculo de estadísticas finales
Unificamos los pares de términos GO para evitar duplicados y calculamos estadísticas consolidadas para cada proteína y cada categoría de término GO.


In [ ]:
def consolidate_pairs(pairs_list):
    unique_pairs = set()
    for pairs in pairs_list:
        unique_pairs.update(pairs)
    return list(unique_pairs)

final_results = results.groupby(['UniProt IDs', 'Category']).agg({
    'Avg_MBL': 'mean',
    'Max_MBL': 'max',
    'GO_Pairs_Max_MBL': lambda x: consolidate_pairs(x)
}).reset_index()

# Visualizamos los resultados finales consolidados para cada categoría y proteína.
print(final_results)

     UniProt IDs Category    Avg_MBL  Max_MBL  \
0     A0A024B7W1        F   3.000000        3   
1     A0A059TC02        P   8.000000       11   
2     A0A061ACU2        P   8.666667       11   
3     A0A075TRK9        C   3.000000        3   
4     A0A075TRK9        F   2.000000        2   
...          ...      ...        ...      ...   
8919      P21599        C   7.000000        7   
8920      P21599        F  10.000000       13   
8921      P21605        F   4.666667        7   
8922      P21605        P   8.533333       15   
8923      P21642        P  11.829268       17   

                                       GO_Pairs_Max_MBL  
0                            [(GO:0008289, GO:0060090)]  
1     [(GO:0010597, GO:0009809), (GO:0007623, GO:001...  
2                            [(GO:0030317, GO:0060279)]  
3                            [(GO:0031012, GO:0005576)]  
4                            [(GO:0016491, GO:0016218)]  
...                                                 ...  
8919 

## Filtrado y visualización de resultados por categoría
Filtramos y visualizamos los resultados finales para cada categoría de término GO (CC, MF, BP).

In [ ]:
results_C = final_results[final_results['Category'] == 'C']
results_F = final_results[final_results['Category'] == 'F']
results_P = final_results[final_results['Category'] == 'P']

# Visualizamos los resultados filtrados por categorías.
print("Resultados para Categoría C:\n", results_C)
print("\nResultados para Categoría F:\n", results_F)
print("\nResultados para Categoría P:\n", results_P)

Resultados para Categoría C:
      UniProt IDs Category    Avg_MBL  Max_MBL  \
3     A0A075TRK9        C   3.000000        3   
9     A0A0D1DWQ2        C   4.000000        4   
33    A0A140H546        C   7.000000        7   
36    A0A144A2H0        C  10.000000       10   
47    A0A1L8F5J9        C  10.000000       10   
...          ...      ...        ...      ...   
8909      P21579        C   6.857143       15   
8911      P21580        C   5.000000        5   
8914      P21583        C   4.000000        6   
8916      P21589        C   2.166667        3   
8919      P21599        C   7.000000        7   

                                       GO_Pairs_Max_MBL  
3                            [(GO:0031012, GO:0005576)]  
9                            [(GO:0140593, GO:0044164)]  
33                           [(GO:0030659, GO:0020005)]  
36                           [(GO:0005737, GO:0020020)]  
47                           [(GO:0017146, GO:0005886)]  
...                              

## Preparación del índice y combinación de resultados
Preparamos un índice usando `all_proteins_dataset.csv`, que incluye todas las proteínas candidatas a no moonlighting que están bajo estudio. Añadimos columnas específicas que almacenan la distancia semántica máxima entre pares de términos GO para las categorías CC y MF, así como los pares que presentan estas distancias máximas.


In [ ]:
# Extraemos y depuramos los identificadores UniProt únicos para evitar duplicados, garantizando
# que cada identificador sea tratado individualmente.
unique_uniprot_ids = all_proteins_metrics['accessioncode'].drop_duplicates().reset_index(drop=True)

# Creamos un DataFrame para manejar únicamente los identificadores UniProt, el cual será utilizado
# a modo de índice para facilitar la combinación de datos.
unique_uniprot_ids_df = pd.DataFrame(unique_uniprot_ids)
unique_uniprot_ids_df.columns = ['UniProt IDs']

# Combinamos los DataFrames results_C y results_F con unique_uniprot_ids_df utilizando la columna 'UniProt IDs',
# e incorporamos las distancias mínimas de rama máximas y los pares de términos GO correspondientes.
unique_uniprot_ids_df.set_index('UniProt IDs', inplace=True)
results_C.set_index('UniProt IDs', inplace=True)
results_F.set_index('UniProt IDs', inplace=True)

final_dataset = unique_uniprot_ids_df.join([
    results_C[['Max_MBL', 'GO_Pairs_Max_MBL']].rename(columns={
        'Max_MBL': 'Max_MBL_CC',
        'GO_Pairs_Max_MBL': 'CC_Pairs_Max_MBL'
    }),
    results_F[['Max_MBL', 'GO_Pairs_Max_MBL']].rename(columns={
        'Max_MBL': 'Max_MBL_MF',
        'GO_Pairs_Max_MBL': 'MF_Pairs_Max_MBL'
    })
], how='left')

# Asignamos un valor predeterminado de 0 en las columnas de distancia máxima donde no existen datos.
final_dataset['Max_MBL_CC'].fillna(0, inplace=True)
final_dataset['Max_MBL_MF'].fillna(0, inplace=True)

# Asignamos un valor predeterminado 'No data' en las columnas que detallan los pares de términos GO
# cuando no existen datos disponibles.
final_dataset['CC_Pairs_Max_MBL'].fillna('No data', inplace=True)
final_dataset['MF_Pairs_Max_MBL'].fillna('No data', inplace=True)

# Reseteamos el índice para convertir 'UniProt IDs' en una columna regular del DataFrame.
final_dataset.reset_index(inplace=True)

# Visualizamos los resultados obtenidos.
print(final_dataset)

     UniProt IDs  Max_MBL_CC            CC_Pairs_Max_MBL  Max_MBL_MF  \
0     A0A024B7W1         0.0                     No data         3.0   
1     A0A059TC02         0.0                     No data         0.0   
2     A0A0B7P9G0         0.0                     No data         0.0   
3     A0A0D2UG83         0.0                     No data         7.0   
4     A0A0G2KTI4         0.0                     No data        11.0   
...          ...         ...                         ...         ...   
4339      P19156         0.0                     No data         3.0   
4340      P19235         4.0  [(GO:0016607, GO:0005886)]         8.0   
4341      P20265         0.0                     No data         7.0   
4342      P20937         4.0  [(GO:0009897, GO:0005886)]         0.0   
4343      P21218         0.0                     No data         6.0   

                MF_Pairs_Max_MBL  
0     [(GO:0008289, GO:0060090)]  
1                        No data  
2                        No da

## Comparación y filtrado de distancias semánticas máximas. Reordenación del dataset.

Determinamos cuál de las distancias semánticas máximas (CC o MF) es mayor para cada proteína y almacenamos este valor mayor en una nueva columna 'Max_Semantic_Distance'. Posteriormente, eliminamos las entradas que no contienen información relevante y ordenamos los resultados para facilitar la interpretación.

In [ ]:
# Calculamos el valor máximo de distancia semántica entre las categorías CC y MF para cada proteína.
final_dataset['Highest_MBL'] = final_dataset[['Max_MBL_CC', 'Max_MBL_MF']].max(axis=1)

# Definimos las condiciones para identificar y eliminar filas que no contienen datos relevantes.
condiciones = (
    (final_dataset['Max_MBL_CC'] == 0) &
    (final_dataset['CC_Pairs_Max_MBL'] == 'No data') &
    (final_dataset['Max_MBL_MF'] == 0) &
    (final_dataset['MF_Pairs_Max_MBL'] == 'No data')
)

# Filtramos el DataFrame para eliminar las filas que cumplen todas las condiciones especificadas anteriormente.
final_dataset_filtered = final_dataset[~condiciones]

# Ordenamos los resultados de manera ascendente según el valor de  'Max_Semantic_Distance'.
sorted_final_dataset = final_dataset_filtered.sort_values('Highest_MBL')

# Seleccionamos las primeras 700 filas del conjunto de datos ordenado.
top_700 = sorted_final_dataset.head(700)

# Visualizamos estos datos.
print("Primeras 700 filas", top_700)

Primeras 700 filas      UniProt IDs  Max_MBL_CC  \
564       O22793         0.0   
3484      G8JZS6         1.0   
3478      G1FNI6         0.0   
3460      C0LGW6         0.0   
3459      C0HLV2         0.0   
...          ...         ...   
1207      O95429         6.0   
3813      P0AFX7         6.0   
1224      O95704         6.0   
3795      P0AC19         0.0   
1165      O95071         6.0   

                                       CC_Pairs_Max_MBL  Max_MBL_MF  \
564                                             No data         1.0   
3484                         [(GO:0009279, GO:0019867)]         0.0   
3478                                            No data         1.0   
3460                                            No data         1.0   
3459                                            No data         1.0   
...                                                 ...         ...   
1207                         [(GO:0005634, GO:0005886)]         0.0   
3813                        

## Enriquecimiento de los datos
Enriquecemos este conjunto de datos con información actualizada y relevante, de manera similar a lo realizado con las proteínas moonlighting en el notebook `data_preparation.ipynb`. Utilizamos datos de UniProt y herramientas de BioPython.


In [ ]:
# Instalamos BioPython.
!pip3 install biopython

# Preparamos los datos.
uniprot_ids = top_700['UniProt IDs'].tolist()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.8 MB/s eta 0:00:00


In [ ]:
from Bio import ExPASy, SwissProt

# Inicializamos diccionarios para almacenar la información recopilada para cada proteína.
pdb_ids = {}
gene_names = {}
protein_names = {}
amino_acid_sequences = {}
organism_names = {}
status_info = {}
protein_existence = {}
go_bp_terms = {}
go_cc_terms = {}
go_mf_terms = {}

# Definimos una función para unir datos y manejar valores que podrían no ser cadenas.
def safe_str_join(data, separator="; "):
  if isinstance(data, list):
    return separator.join(str(item) for item in data)
  elif isinstance(data, dict):
    return separator.join(f"{key}: {value}" for key, value in data.items())
  elif data is None:
    return "N/A"
  return str(data)

# Iteramos sobre cada identificador UniProt para realizar consultas y recopilar información.
for uniprot_id in uniprot_ids:
  try:
    handle = ExPASy.get_sprot_raw(uniprot_id)
    record = SwissProt.read(handle)

    # Almacenamos la información específica obtenida de cada registro.
    pdb_entries = [safe_str_join(ref[1]) for ref in record.cross_references if ref[0] == 'PDB']
    pdb_ids[uniprot_id] = safe_str_join(pdb_entries)
    gene_names[uniprot_id] = safe_str_join(record.gene_name, ", ")
    protein_names[uniprot_id] = safe_str_join(record.description.split("=")[1].split(";")[0])
    amino_acid_sequences[uniprot_id] = safe_str_join(record.sequence)
    organism_names[uniprot_id] = safe_str_join(record.organism)
    status_info[uniprot_id] = "Reviewed" if record.data_class == "Reviewed" else "Unreviewed"
    protein_existence[uniprot_id] = safe_str_join(record.protein_existence)

    # Procesamos los términos GO asociados a cada proteína, clasificándolos según su categoría.
    go_terms_processing = lambda ref, code: [f"{r[1]} - {r[2][2:]}" for r in ref if r[0] == "GO" and r[2].startswith(code)]
    go_bp_terms[uniprot_id] = safe_str_join(go_terms_processing(record.cross_references, "P:"))
    go_cc_terms[uniprot_id] = safe_str_join(go_terms_processing(record.cross_references, "C:"))
    go_mf_terms[uniprot_id] = safe_str_join(go_terms_processing(record.cross_references, "F:"))

  except Exception as e:
    print(f"Error al procesar el identificador {uniprot_id}: {e}")
    pdb_ids[uniprot_id] = gene_names[uniprot_id] = protein_names[uniprot_id] = amino_acid_sequences[uniprot_id] = organism_names[uniprot_id] = go_bp_terms[uniprot_id] = go_cc_terms[uniprot_id] = go_mf_terms[uniprot_id] = status_info[uniprot_id] = protein_existence[uniprot_id] = "Error al recuperar datos"

# Actualizamos el DataFrame con los datos recopilados.
top_700['PDB ID'] = top_700['UniProt IDs'].map(pdb_ids)
top_700['Gene Name'] = top_700['UniProt IDs'].map(gene_names)
top_700['Protein Name'] = top_700['UniProt IDs'].map(protein_names)
top_700['Amino Acid Sequence'] = top_700['UniProt IDs'].map(amino_acid_sequences)
top_700['Organism Name'] = top_700['UniProt IDs'].map(organism_names)
top_700['Status'] = top_700['UniProt IDs'].map(status_info)
top_700['Protein Existence'] = top_700['UniProt IDs'].map(protein_existence)
top_700['GO BP Terms'] = top_700['UniProt IDs'].map(go_bp_terms)
top_700['GO CC Terms'] = top_700['UniProt IDs'].map(go_cc_terms)
top_700['GO MF Terms'] = top_700['UniProt IDs'].map(go_mf_terms)

top_700_final = top_700[['UniProt IDs', 'PDB ID', 'Gene Name', 'Protein Name', 'Amino Acid Sequence', 'Organism Name', 'Status', 'Protein Existence', 'GO BP Terms', 'GO CC Terms', 'GO MF Terms', 'Max_MBL_CC', 'CC_Pairs_Max_MBL', 'Max_MBL_MF', 'MF_Pairs_Max_MBL', 'Highest_MBL']]

# Visualizamos las primeras filas del DataFrame obtenido.
top_700_final.head()

<ipython-input-13-25f0131c791f>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_700['PDB ID'] = top_700['UniProt IDs'].map(pdb_ids)
<ipython-input-13-25f0131c791f>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_700['Gene Name'] = top_700['UniProt IDs'].map(gene_names)
<ipython-input-13-25f0131c791f>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,UniProt IDs,PDB ID,Gene Name,Protein Name,Amino Acid Sequence,Organism Name,Status,Protein Existence,GO BP Terms,GO CC Terms,GO MF Terms,Max_MBL_CC,CC_Pairs_Max_MBL,Max_MBL_MF,MF_Pairs_Max_MBL,Highest_MBL
564,O22793,5YDG,{'Name': 'MORF2 {ECO:0000303|PubMed:22411807}'...,"Multiple organellar RNA editing factor 2, chlo...",MALPLSGTRHLTRALLSNVTLMAPPRIPSSVHYGGSRLGCSTRFFS...,Arabidopsis thaliana (Mouse-ear cress).,Reviewed,1,GO:0016554 - cytidine to uridine editing; GO:0...,GO:0009507 - chloroplast; GO:0005783 - endopla...,GO:0046983 - protein dimerization activity; GO...,0.0,No data,1.0,"[(GO:0046983, GO:0042803)]",1.0
3484,G8JZS6,4FE9,"{'Name': 'susF', 'OrderedLocusNames': ['BT_369...",Outer membrane protein SusF,MKKHLIYTGMFLAAIGFSACNEDFKDWADPQSNPQEESAGQLTATF...,Bacteroides thetaiotaomicron (strain ATCC 2914...,Reviewed,1,GO:0005983 - starch catabolic process; GO:0005...,GO:0009279 - cell outer membrane; GO:0019867 -...,GO:2001070 - starch binding,1.0,"[(GO:0009279, GO:0019867)]",0.0,No data,1.0
3478,G1FNI6,5A10; 5A11,{'Name': 'TFP {ECO:0000303|PubMed:21783213}'},N-(sulfonatooxy)prop-2-enimidothioate sulfolya...,MARTLQGEWMKVEQKGGQVPAPRSSHGIAVIGDKLYCFGGEDPPYE...,Thlaspi arvense (Field penny-cress).,Reviewed,1,GO:0019760 - glucosinolate metabolic process; ...,GO:0005829 - cytosol; GO:0005634 - nucleus,GO:0030234 - enzyme regulator activity; GO:004...,0.0,No data,1.0,"[(GO:0042802, GO:0042803)]",1.0
3460,C0LGW6,5XJO; 5XJX; 5XKJ,"{'Name': 'ERL1 {ECO:0000303|PubMed:14985254}',...",LRR receptor-like serine/threonine-protein kin...,MKEKMQRMVLSLAMVGFMVFGVASAMNNEGKALMAIKGSFSNLVNM...,Arabidopsis thaliana (Mouse-ear cress).,Reviewed,1,GO:0009553 - embryo sac development; GO:001631...,GO:0016020 - membrane; GO:0005886 - plasma mem...,GO:0005524 - ATP binding; GO:0106310 - protein...,0.0,No data,1.0,"[(GO:0033612, GO:0005102)]",1.0
3459,C0HLV2,7ZU8; 7ZVA; 7ZVB; 7ZVC,,Protein neprosin {ECO:0000303|PubMed:27481162},MQAKFFTFVILSSVFYFNYPLAEARSIQARLANKPKGTIKTIKGDD...,Nepenthes x ventrata (Red tropical pitcher pla...,Reviewed,1,GO:0006508 - proteolysis,GO:0005576 - extracellular region,GO:0004175 - endopeptidase activity; GO:007001...,0.0,No data,1.0,"[(GO:0004175, GO:0070012)]",1.0


## Etiquetado y almacenamiento de los resultados finales
Añadimos una nueva columna 'Class' al DataFrame, asignando valor 'False' a todas las entradas para reflejar que todas las proteínas del conjunto de datos son no moonlighting. Finalmente, guardamos los resultados en un archivo CSV, que será objeto de futuros análisis.

In [ ]:
# Añadimos la columna 'Class'.
top_700_final['Class'] = False

# Guardamos el DataFrame obtenido.
top_700_final.to_csv('/content/drive/My Drive/top700_stats_dataset.csv', index=False)

# Visualizamos los primeros registros del nuevo DataFrame.
print(top_700_final.head())

<ipython-input-14-fa75d57d8aa8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_700_final['Class'] = False


     UniProt IDs                  PDB ID  \
564       O22793                    5YDG   
3484      G8JZS6                    4FE9   
3478      G1FNI6              5A10; 5A11   
3460      C0LGW6        5XJO; 5XJX; 5XKJ   
3459      C0HLV2  7ZU8; 7ZVA; 7ZVB; 7ZVC   

                                              Gene Name  \
564   {'Name': 'MORF2 {ECO:0000303|PubMed:22411807}'...   
3484  {'Name': 'susF', 'OrderedLocusNames': ['BT_369...   
3478      {'Name': 'TFP {ECO:0000303|PubMed:21783213}'}   
3460  {'Name': 'ERL1 {ECO:0000303|PubMed:14985254}',...   
3459                                                      

                                           Protein Name  \
564   Multiple organellar RNA editing factor 2, chlo...   
3484                        Outer membrane protein SusF   
3478  N-(sulfonatooxy)prop-2-enimidothioate sulfolya...   
3460  LRR receptor-like serine/threonine-protein kin...   
3459     Protein neprosin {ECO:0000303|PubMed:27481162}   

                         